In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten,SimpleRNN
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import load_model

In [5]:
word_index = imdb.get_word_index()
index_word = {v : k for k, v in word_index.items()}

In [6]:
model = load_model('imdb_simpleRNN_model.h5')

In [7]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 664,835 (2.54 MB)

 Trainable params: 664,833 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [9]:
model.get_weights()

[array([[-0.19600874, -0.11862294,  0.09832422, ...,  0.18856609,
         -0.13580643, -0.15805525],
        [ 0.00039762, -0.0436274 , -0.03969174, ..., -0.00337445,
         -0.07321288, -0.03656724],
        [-0.02696418,  0.01238322,  0.00877474, ...,  0.06468385,
         -0.0139844 ,  0.05433552],
        ...,
        [ 0.08685664,  0.05990611, -0.00698751, ..., -0.04502603,
          0.02854525,  0.06161044],
        [-0.09308556, -0.11821822,  0.10909813, ...,  0.12863722,
          0.05540824, -0.03743386],
        [ 0.05021458,  0.06502638, -0.01310647, ..., -0.02582034,
          0.00454476,  0.02484106]], shape=(10000, 64), dtype=float32),
 array([[ 0.18271059, -0.21829596, -0.12575829, ...,  0.17596601,
         -0.1178619 ,  0.3113478 ],
        [ 0.1927614 , -0.15805605, -0.12678453, ...,  0.04978741,
          0.16561593,  0.09654053],
        [ 0.02795538,  0.1640955 ,  0.0707346 , ...,  0.10526812,
          0.09569184,  0.0546166 ],
        ...,
        [-0.03157074

In [10]:
def decode(text):
    return ' '.join([index_word.get(i - 3, '?') for i in text])

In [11]:
def preprocess_review(review):
    
    tokens = review.lower().split()
    
    indices = [word_index.get(word, 2) + 3 for word in tokens]  # 2 is for unknown words
    
    padded_sequence = pad_sequences([indices], maxlen=500)
    return padded_sequence

In [12]:
def predict_review(review):
    # Preprocess the review
    review_seq = []
    for word in review.split():
        index = word_index.get(word, 2)  # 2 is for unknown words
        review_seq.append(index + 3)  # Offset by 3 as per Keras IMDB dataset

    review_pad = pad_sequences([review_seq], maxlen=500)

    # Predict sentiment
    prediction = model.predict(review_pad)
    sentiment = "Positive" if prediction[0][0] >= 0.5 else "Negative"
    return sentiment, prediction[0][0]

In [14]:
# testing the function
sample_review = "The movie was fantastic with brilliant performances and a gripping storyline."
sentiment, confidence = predict_review(sample_review)

print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Review: The movie was fantastic with brilliant performances and a gripping storyline.
Predicted Sentiment: Positive (Confidence: 1.8826)
